# CogVideoX Correlating Videos for a Continuous 30 Second Video

**This tutorial is available in English and is attached below the Chinese explanation**

这篇内容通过统一视频生成场景和内容的方式将五段CogVideoX生成的6秒短视频相关联起来，得到一个大致流畅的30秒长视频。第一段视频以用户自定义的prompt来进行文生视频。后四段视频则以前一段视频的最后一帧为基础，以及GLM-4V对此帧的下一步预判进行图生视频，从而建立视频之间联系并得到大致连贯的长视频。

This content relates five CogVideoX-generated 6-second short videos by unifying the generation context and content to get a roughly smooth 30-second long video. The first video is text-generated video with a user-defined prompt. The last four videos are graphically generated based on the last frame of the previous video and GLM-4V's next prediction of this frame, thus establishing connections between the videos and obtaining a roughly coherent long video.


In [1]:
import os
from zhipuai import ZhipuAI

os.environ["ZHIPUAI_API_KEY"] = "your api key"
client = ZhipuAI()

## 2. Prepare the Methods for Generating Videos

现在，我们需要准备一些生成连贯视频的方法。

Now, we need to prepare some methods to generate coherent videos.

这个方法根据给定的prompt使用 CogVideoX 生成视频，并标注一个索引，代表其在视频系列中的位置。利用重试机制，此方法在视频生成后立即找回并保存视频。然后，它使用MoviePy提取并保存视频的最后一帧，供下一个视频参考。

This method generates a video using CogVideoX based on the given prompt and is labelled with an index that represents its position in the series of videos. Utilizing a retry mechanism, this method retrieves and saves the video as soon the video is generated. Then, it extracts and saves the last frame of the video using MoviePy for the next video to reference. 

In [2]:
import requests
import time
from moviepy.editor import VideoFileClip
from PIL import Image
import numpy as np

def generate_video_and_extract_frame(prompt, video_index):
    response = client.videos.generations(
        model="cogvideox",
        prompt=prompt
    )
    video_id = response.id

    max_attempts = 30
    attempt = 0

    while attempt < max_attempts:
        retrieval_response = client.videos.retrieve_videos_result(id=video_id)
        if retrieval_response.task_status == "SUCCESS":
            break
        elif retrieval_response.task_status == "FAILED":
            raise Exception(f"Video generation failed for video {video_index}")
        time.sleep(10)
        attempt += 1

    if attempt == max_attempts:
        raise TimeoutError(f"Timeout while waiting for video {video_index} generation")

    video_result = retrieval_response.video_result[0] if retrieval_response.video_result else None

    video_url = video_result.url
    video_response = requests.get(video_url)

    video_filename = f"data/video_{video_index}.mp4"
    with open(video_filename, "wb") as f:
        f.write(video_response.content)

    clip = VideoFileClip(video_filename)
    duration = clip.duration
    last_frame = clip.get_frame(duration)
    img = Image.fromarray(np.uint8(last_frame))
    last_frame_filename = "data/last_frame.png"
    img.save(last_frame_filename)

    return video_filename, last_frame_filename

这个方法使用 ZhipuAI 的另一个模型 GLM-4V 对视频的最后一帧进行分析，并生成下一个视频的合理下一步内容。为了实现这一目标，该模型会接收最后一帧的 base64 字符串编码，以及要求对接下来可能发生的事情进行合理预测的问题，然后输出最后一帧的下一步内容。

This method uses GLM-4V, another model from ZhipuAI, to analyze the last frame and give a reasonable next step for the generation of the next video. To achieve this, the model takes in the base64 string encoding of this last frame as well as the question asking for a reasonable prediction of what could happen next, then, outputs a response of the next step of the last frame.

In [3]:
import io
import base64

def analyze_last_frame(last_frame_path):
    with Image.open(last_frame_path) as image:
        buffered = io.BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What could happen next? Give one possibility."
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": img_str
                    }
                }
            ]
        }
    ]

    response = client.chat.completions.create(
        model="glm-4v",
        messages=messages,
        temperature=0.1,
        top_p=0.1
    )
    return response.choices[0].message.content

下一步，这两个方法分别是根据现有视频生成下一个视频和拼接全部视频。为了保证前后两个视频的关联性，下一个视频的prompt为上个视频的prompt和GLM-4V给出的下一步预判的拼接。最后，把五个视频拼接起来并保存。

In the next step, the two methods are generating the next video and concatenating the whole video based on the existing video, respectively. In order to ensure the videos are related, the prompt of the next video is a concatenation of the prompt of the previous video and the next prediction given by GLM-4V. Finally, the five videos are stitched and saved.

In [4]:
def generate_next_prompt(last_frame_path, previous_prompt):
    next_prompt = previous_prompt + analyze_last_frame(last_frame_path)
    return next_prompt

from moviepy.editor import VideoFileClip, concatenate_videoclips

def stitch_videos(video_filenames):
    clips = [VideoFileClip(filename) for filename in video_filenames]
    final_clip = concatenate_videoclips(clips)
    final_clip.write_videofile("data/final_video_Labrador.mp4")

## 3. Generate the continuous videos

最后，编写初始的prompt并实施前面的方法，以获得我们的30秒长视频。视频应基本连贯，多次生成将提高视频质量。

Lastly, write the initial prompt and implement the methods to get our 30 second long video. It should be mostly coherent, generating multiple times would enhance the video quality.

In [5]:
initial_prompt = "A Labrador is walking in the rain"

video_filenames = []
previous_prompt = initial_prompt

for i in range(5):
    video_filename, last_frame_filename = generate_video_and_extract_frame(previous_prompt, i+1)
    video_filenames.append(video_filename)
    if i < 4:
        previous_prompt = generate_next_prompt(last_frame_filename, previous_prompt)

stitch_videos(video_filenames)

Moviepy - Building video final_video_Labrador.mp4.
Moviepy - Writing video final_video_Labrador.mp4



Moviepy - Done !
Moviepy - video ready final_video_Labrador.mp4
